> ⚠️ **작업 중 (Work in Progress)**: 이 노트북은 현재 개발 중입니다. 일부 코드가 불완전하거나 변경될 수 있습니다.

## 📋 목차

- [평가 개요](#평가-개요)
- [환경 설정](#환경-설정)
- [평가 기준 이해](#평가-기준-이해)
- [평가 실행 및 결과 분석](#평가-실행-및-결과-분석)
- [평가 모범 사례](#평가-모범-사례)

## 🎯 학습 목표

- AI 에이전트 평가의 중요성 이해
- Foundry의 자동 평가 기능 활용
- 다양한 평가 지표의 의미와 활용법 학습
- 합성 데이터를 사용한 평가 수행
- 평가 결과 해석 및 개선 방안 도출

## ⏱️ 예상 소요 시간

약 10분

## 평가 개요

### 왜 평가가 중요한가?

AI 에이전트를 프로덕션에 배포하기 전에 다음 사항을 검증해야 합니다:

```
정확성 → 관련성 → 일관성 → 자연스러움 → 안전성
```

평가 없이 배포하면:
- ❌ 부정확한 답변으로 사용자 신뢰 저하
- ❌ 관련 없는 응답으로 사용자 경험 악화
- ❌ 일관성 없는 품질로 브랜드 이미지 손상
- ❌ 부적절한 콘텐츠 생성으로 법적 문제

### Microsoft Foundry의 평가 기능

Foundry는 다음을 자동화합니다:
- ✅ 테스트 데이터 생성 (Synthetic generation)
- ✅ 다양한 평가 지표 적용
- ✅ 대규모 평가 실행
- ✅ 결과 시각화 및 분석

## 환경 설정

평가를 위한 환경을 설정합니다.

In [ ]:
# 환경 변수 로드
import json
import os
import subprocess

# PATH 환경변수 설정 (Azure CLI를 찾을 수 있도록)
possible_paths = [
    "/opt/homebrew/bin",  # macOS (Apple Silicon)
    "/usr/local/bin",     # macOS (Intel) / Linux
    "/usr/bin",           # Linux / GitHub Codespaces
    "/home/linuxbrew/.linuxbrew/bin"  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
except:
    pass

paths_to_add = []
if az_path and az_path not in os.environ.get("PATH", ""):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get("PATH", ""):
            paths_to_add.append(path)

if paths_to_add:
    new_path = ":".join(paths_to_add) + ":" + os.environ.get("PATH", "")
    os.environ["PATH"] = new_path

# 이전 노트북에서 저장한 설정 파일 로드
config_file = ".foundry_config.json"
try:
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    # 환경 변수 설정
    FOUNDRY_NAME = config.get("FOUNDRY_NAME")
    RESOURCE_GROUP = config.get("RESOURCE_GROUP")
    LOCATION = config.get("LOCATION")
    TENANT_ID = config.get("TENANT_ID")
    PROJECT_NAME = config.get("PROJECT_NAME", "proj-default")
    PROJECT_ENDPOINT = config.get("FOUNDRY_ENDPOINT")
    
    # 환경 변수로도 설정 (다른 도구들이 사용할 수 있도록)
    os.environ["FOUNDRY_NAME"] = FOUNDRY_NAME
    os.environ["LOCATION"] = LOCATION
    os.environ["RESOURCE_GROUP"] = RESOURCE_GROUP
    os.environ["AZURE_SUBSCRIPTION_ID"] = config.get("AZURE_SUBSCRIPTION_ID", "")
    os.environ["_ENDPOINT"] = PROJECT_ENDPOINT
    
    print(f"✅ 설정 파일 '{config_file}'에서 환경 변수를 로드했습니다.")
    print(f"\n📌 Foundry Name: {FOUNDRY_NAME}")
    print(f"📌 Resource Group: {RESOURCE_GROUP}")
    print(f"📌 Location: {LOCATION}")
    print(f"📌 프로젝트 엔드포인트: {PROJECT_ENDPOINT}")
    
except FileNotFoundError:
    print(f"⚠️ '{config_file}' 파일을 찾을 수 없습니다.")
    print("💡 01-setup.ipynb를 먼저 실행하여 환경을 설정하세요.")
    raise

# 필수 패키지 설치
%pip install -q azure-ai-evaluation azure-ai-projects azure-identity

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

print(f"\n💡 사용할 프로젝트 엔드포인트: {PROJECT_ENDPOINT}")


In [ ]:
# 간단한 에이전트 평가 예제
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

# 테스트 데이터 준비
test_queries = [
    "Python에서 리스트를 정렬하는 방법은?",
    "머신러닝과 딥러닝의 차이점을 설명해주세요.",
    "서울의 인구는 몇 명인가요?",
    "최근 AI 기술 동향을 알려주세요.",
    "클라우드 컴퓨팅의 장점은 무엇인가요?"
]

print("📊 에이전트 테스트 시작...\n")
print("=" * 80)

# 자동으로 ModelRouterAgent 찾기
try:
    agents = list(project_client.agents.list())
    agent_router = next((a for a in agents if a.name == "ModelRouterAgent"), None)
    
    if not agent_router:
        print("⚠️ 'ModelRouterAgent'를 찾을 수 없습니다.")
        print("💡 03-agents.ipynb를 먼저 실행하여 에이전트를 생성하세요.")
        raise ValueError("Agent not found")
    
    print(f"✅ 사용할 에이전트: {agent_router.name} (ID: {agent_router.id})\n")
    
    # 데모용 Mock 응답 (실제 환경에서는 Portal 평가 권장)
    print("\n💡 참고: SDK 버전 제한으로 인해 Mock 응답을 사용합니다.")
    print("   실제 대규모 평가는 Azure Portal을 사용하세요.\n")
    
    mock_responses = [
        "Python에서 리스트를 정렬하려면 `sort()` 메서드나 `sorted()` 함수를 사용할 수 있습니다. `list.sort()`는 리스트를 제자리에서 정렬하고, `sorted(list)`는 새로운 정렬된 리스트를 반환합니다.",
        "머신러닝은 명시적 프로그래밍 없이 데이터에서 패턴을 학습하는 기술이며, 딥러닝은 인공 신경망을 사용하는 머신러닝의 한 분야입니다. 딥러닝은 더 복잡한 패턴을 학습할 수 있지만 더 많은 데이터와 컴퓨팅 파워가 필요합니다.",
        "서울의 인구는 약 950만 명입니다. (2023년 기준)",
        "최근 AI 기술 동향으로는 대규모 언어 모델(LLM)의 발전, 멀티모달 AI, 생성형 AI의 확산, AI 에이전트 시스템의 발전 등이 있습니다.",
        "클라우드 컴퓨팅의 주요 장점은 확장성, 비용 효율성, 접근성, 자동 업데이트, 재해 복구 기능 등입니다. 필요에 따라 리소스를 유연하게 조정할 수 있어 효율적입니다."
    ]
    
    responses = []
    for i, query in enumerate(test_queries, 1):
        print(f"\n[질문 {i}/{len(test_queries)}]: {query}")
        response = mock_responses[i-1]
        responses.append({"query": query, "response": response})
        print(f"[응답]: {response[:100]}...")
    
    print("\n" + "=" * 80)
    print(f"\n✅ {len(test_queries)}개 질문 테스트 완료!")
    print(f"\n💡 포털에서 더 자세한 평가를 진행하세요:")
    print("   https://ai.azure.com > Build > Evaluations")

except Exception as e:
    print(f"\n⚠️ 에러 발생: {e}")
    print("\n💡 해결 방법:")
    print("   1. 03-agents.ipynb를 먼저 실행하여 ModelRouterAgent를 생성")
    print("   2. Azure에 로그인했는지 확인 (az login)")
    print("   3. FOUNDRY_NAME이 올바른지 확인")

### Azure AI Evaluation SDK 사용하기

SDK를 통해 프로그래밍 방식으로 평가를 실행할 수 있습니다. 다양한 Evaluator를 사용하여 에이전트 응답의 품질을 측정합니다.

In [ ]:
# Azure AI Evaluation SDK Evaluator 임포트
from azure.ai.evaluation import (
    CoherenceEvaluator,
    FluencyEvaluator,
    GroundednessEvaluator,
    RelevanceEvaluator,
)

# Azure OpenAI 모델 설정 (평가용)
model_config = {
    "azure_endpoint": f"https://{FOUNDRY_NAME}.openai.azure.com/",
    "api_version": "2024-08-01-preview",
    "azure_deployment": "gpt-5.1",  # 평가에 사용할 모델 (하이픈 포함)
}

# Evaluator 인스턴스 생성
coherence_evaluator = CoherenceEvaluator(model_config=model_config)
fluency_evaluator = FluencyEvaluator(model_config=model_config)
groundedness_evaluator = GroundednessEvaluator(model_config=model_config)
relevance_evaluator = RelevanceEvaluator(model_config=model_config)

print("✅ Evaluator 인스턴스가 생성되었습니다:")
print("   - CoherenceEvaluator: 논리적 일관성 평가")
print("   - FluencyEvaluator: 자연스러움 평가")
print("   - GroundednessEvaluator: 사실 기반 평가")
print("   - RelevanceEvaluator: 관련성 평가")

In [ ]:
# 평가 실행 예제
# 위에서 수집한 responses 데이터를 사용하여 평가합니다

print("📊 SDK 기반 평가 실행 중...\n")
print("=" * 80)

evaluation_results = []

for i, item in enumerate(responses, 1):
    query = item["query"]
    response = item["response"]
    
    # 평가용 컨텍스트 (실제 환경에서는 RAG에서 가져온 문서 사용)
    context = "일반적인 지식 기반 질문에 대한 응답입니다."
    
    print(f"\n[샘플 {i}/{len(responses)}]")
    print(f"질문: {query[:50]}...")
    
    # 각 Evaluator로 평가
    coherence_score = coherence_evaluator(query=query, response=response)
    fluency_score = fluency_evaluator(query=query, response=response)
    groundedness_score = groundedness_evaluator(query=query, response=response, context=context)
    relevance_score = relevance_evaluator(query=query, response=response, context=context)
    
    result = {
        "query": query,
        "response": response[:100],
        "coherence": coherence_score.get("coherence", "N/A"),
        "fluency": fluency_score.get("fluency", "N/A"),
        "groundedness": groundedness_score.get("groundedness", "N/A"),
        "relevance": relevance_score.get("relevance", "N/A"),
    }
    evaluation_results.append(result)
    
    print(f"  Coherence: {result['coherence']}/5")
    print(f"  Fluency: {result['fluency']}/5")
    print(f"  Groundedness: {result['groundedness']}/5")
    print(f"  Relevance: {result['relevance']}/5")

print("\n" + "=" * 80)
print(f"\n✅ {len(responses)}개 샘플 평가 완료!")

In [ ]:
# 평가 결과 요약
import statistics

print("=" * 80)
print("📈 평가 결과 요약")
print("=" * 80)

# 각 지표별 평균 계산
metrics = ["coherence", "fluency", "groundedness", "relevance"]
averages = {}

for metric in metrics:
    scores = [r[metric] for r in evaluation_results if isinstance(r[metric], (int, float))]
    if scores:
        averages[metric] = statistics.mean(scores)
    else:
        averages[metric] = "N/A"

print(f"\n📊 Overall Scores:")
print(f"   Coherence:    {averages['coherence']:.2f}/5.0" if isinstance(averages['coherence'], float) else f"   Coherence:    {averages['coherence']}")
print(f"   Fluency:      {averages['fluency']:.2f}/5.0" if isinstance(averages['fluency'], float) else f"   Fluency:      {averages['fluency']}")
print(f"   Groundedness: {averages['groundedness']:.2f}/5.0" if isinstance(averages['groundedness'], float) else f"   Groundedness: {averages['groundedness']}")
print(f"   Relevance:    {averages['relevance']:.2f}/5.0" if isinstance(averages['relevance'], float) else f"   Relevance:    {averages['relevance']}")

# Pass/Fail 기준 (4.0 이상이면 Pass)
threshold = 3.5
passed = sum(1 for r in evaluation_results 
             if all(isinstance(r[m], (int, float)) and r[m] >= threshold for m in metrics))
pass_rate = (passed / len(evaluation_results)) * 100 if evaluation_results else 0

print(f"\n✅ Pass Rate: {pass_rate:.1f}% ({passed}/{len(evaluation_results)} samples)")
print(f"   (기준: 모든 지표 ≥ {threshold})")

print("\n" + "=" * 80)
print("\n💡 권장 사항:")
if averages.get('groundedness', 5) < 4.0:
    print("   ⚠️ Groundedness 개선 필요: Knowledge Base 보강 또는 Instructions 수정")
if averages.get('relevance', 5) < 4.0:
    print("   ⚠️ Relevance 개선 필요: 에이전트 Instructions를 더 명확하게")
if averages.get('coherence', 5) < 4.0:
    print("   ⚠️ Coherence 개선 필요: 응답 구조화 가이드라인 추가")
if averages.get('fluency', 5) < 4.0:
    print("   ⚠️ Fluency 개선 필요: 모델 업그레이드 또는 프롬프트 개선")
if pass_rate >= 80:
    print("   ✅ 전체적으로 양호한 성능입니다!")

# 결과 시각화 (선택사항)
try:
    import matplotlib.pyplot as plt
    
    # 지표별 평균 점수 막대 그래프
    valid_metrics = {k: v for k, v in averages.items() if isinstance(v, (int, float))}
    
    if valid_metrics:
        plt.figure(figsize=(10, 6))
        plt.bar(valid_metrics.keys(), valid_metrics.values())
        plt.axhline(y=threshold, color='r', linestyle='--', label=f'Threshold ({threshold})')
        plt.ylim(0, 5)
        plt.ylabel('Score')
        plt.title('Evaluation Results Summary')
        plt.legend()
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        print("\n📊 시각화 완료!")
except ImportError:
    print("\n💡 시각화를 위해 matplotlib 설치: pip install matplotlib")

### Foundry 제공 Evaluator 전체 목록

Foundry는 6개 카테고리, 32개의 Evaluator를 제공합니다:

| 카테고리 | Evaluator 예시 |
|---------|---------------|
| **일반 품질** | CoherenceEvaluator, FluencyEvaluator, QAEvaluator |
| **텍스트 유사도** | SimilarityEvaluator, F1ScoreEvaluator, BleuScoreEvaluator |
| **RAG** | GroundednessEvaluator, RelevanceEvaluator, RetrievalEvaluator |
| **에이전트** | IntentResolutionEvaluator, TaskAdherenceEvaluator, ToolCallAccuracyEvaluator |
| **위험/안전** | ViolenceEvaluator, SexualEvaluator, ContentSafetyEvaluator |
| **Azure OpenAI Graders** | AzureOpenAILabelGrader, AzureOpenAIGrader |

자세한 내용은 [Azure AI Evaluation 문서](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-evaluators)를 참조하세요.

## 평가 기준 이해

Foundry는 다음 4가지 핵심 평가 기준을 제공합니다:

| 기준 | 설명 |
|------|------|
| **Groundedness** | 응답이 제공된 컨텍스트/지식에 기반하는지 |
| **Relevance** | 응답이 질문과 관련이 있는지 |
| **Coherence** | 응답이 논리적으로 일관성 있는지 |
| **Fluency** | 응답이 자연스럽고 문법적으로 올바른지 |

# 간단한 에이전트 평가 예제
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

# 테스트 데이터 준비
test_queries = [
    "Python에서 리스트를 정렬하는 방법은?",
    "머신러닝과 딥러닝의 차이점을 설명해주세요.",
    "서울의 인구는 몇 명인가요?",
    "최근 AI 기술 동향을 알려주세요.",
    "클라우드 컴퓨팅의 장점은 무엇인가요?"
]

print("📊 에이전트 테스트 시작...\n")
print("=" * 80)

# 에이전트 응답 수집
agent_id = "<your-agent-id>"  # ⚠️ 03-agents에서 생성한 에이전트 ID

responses = []
for i, query in enumerate(test_queries, 1):
    print(f"\n[질문 {i}/{len(test_queries)}]: {query}")
    
    # Thread 생성
    thread = project_client.agents.create_thread()
    
    # 메시지 전송
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=query
    )
    
    # Run 실행
    run = project_client.agents.create_and_process_run(
        thread_id=thread.id,
        assistant_id=agent_id
    )
    
    # 응답 수집
    messages = project_client.agents.list_messages(thread_id=thread.id)
    response = messages.data[0].content[0].text.value
    
    responses.append({"query": query, "response": response})
    print(f"[응답]: {response[:100]}...")
    
print("\n" + "=" * 80)
print(f"\n✅ {len(test_queries)}개 질문 테스트 완료!")
print(f"\n💡 포털에서 더 자세한 평가를 진행하세요:")
print("   https://ai.azure.com > Build > Evaluations")

### 각 기준이 중요한 이유

| Groundedness | Relevance | Coherence | Fluency |
|--------------|-----------|-----------|----------|
| 사용자 신뢰 확보 | 사용자 만족도 향상 | 이해하기 쉬운 답변 | 사용자 경험 향상 |
| 법적 책임 최소화 | 효율적 정보 전달 | 전문적 이미지 | 브랜드 이미지 유지 |
| 허위 정보 방지 | 대화 흐름 유지 | 신뢰성 향상 | 이해도 증가 |

## 평가 실행 및 결과 분석

### 평가 실행

1. **Submit** 버튼 클릭
2. 평가가 백그라운드에서 실행됩니다 (약 10-15분)
3. Evaluations 페이지에서 진행 상황 확인

### 결과 해석

### 대규모 평가는 Portal 사용 권장

**SDK vs Portal 비교:**

| 기능 | Python SDK | Azure Portal |
|------|-----------|--------------|
| 샘플 수 | 소규모 (10-20) | 대규모 (50-200+) |
| 시각화 | 제한적 | 풍부한 대시보드 |
| Synthetic 데이터 | 수동 생성 | 자동 생성 |




















| **Human Evaluation** | 자동 평가와 병행하여 새로운 문제 패턴 발견 || **기준선 점수** | Groundedness ≥4.0 / 나머지 ≥3.5 / Pass rate ≥80% || **평가 주기** | 개발 중: 매 업데이트 / 배포 전: 필수 / 배포 후: 주간/월간 || **테스트 시나리오** | 일반·복잡·모호·다국어 질문 + Edge cases || **샘플 수** | 개발: 10-20개 / 테스트: 50-100개 / 프로덕션: 200+개 ||------|----------|| 항목 | 권장 사항 |### 권장 사항6. Submit → 결과 확인 (10-15분 소요)5. Criteria: Groundedness, Relevance, Coherence, Fluency 선택4. Data: Synthetic generation (50-200 샘플)3. Target: Agent 선택 (예: ModelRouterAgent)2. + Create new evaluation 클릭1. https://ai.azure.com → Build → Evaluations**Portal에서 평가하는 방법:**| 협업 | 어려움 | 팀 공유 가능 || 결과 관리 | 로컬 저장 | 클라우드 저장 및 공유 |Pass Rate: 85% (43/50 samples)
```

**개선이 필요한 영역:**
- Groundedness가 4.0 이하인 샘플 검토
- 실패한 7개 샘플 분석
- Instructions 개선 또는 Knowledge Base 보강

## 📚 추가 리소스

- [Azure AI Evaluation 개요](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/observability?view=foundry#what-are-evaluators)
- [파운드리 포털에서 평가 실행](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/evaluate-generative-ai-app?view=foundry)
- [에이전트 평가](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-evaluators/agent-evaluators?view=foundry)

## 📚 추가 리소스

- [Azure AI Evaluation 개요](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/observability?view=foundry#what-are-evaluators)
- [파운드리 포털에서 평가 실행](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/evaluate-generative-ai-app?view=foundry)
- [에이전트 평가](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-evaluators/agent-evaluators?view=foundry)